# 实例7：用Python操作Word批量生成合同

我们在实例5中演示了如何用Python批量生成word版邀请函。我们是简单粗暴地找到需要填写受邀者信息所在位置（即run），然后将这个run直接替换成受邀者的公司名及姓名。因为只有一处需要替换，所以这个方法行得通，但遇到合同，一般有十来处需要修改，如果也逐个去找其位置所在的run，那就反而会降低我们的工作效率，背离办公自动化的初衷了。实例5可以作为入门`python-docx`模块的练手项目。

对于合同的批量处理，我们将使用更聪明的办法。我们的思路是，先建立一个word模板，在合同里面需要变动信息的地方用“【....】”来代替，比如“【合同编号】”等。然后再建一个Excel文档，将“【合同编号】”等信息作为标题，将不同的合同信息放入这个Excel的每一行。然后用`python-docx`去读取word模板中的所有内容，凡是遇到“【....】”的字符，就用Excel中的对应标题下的信息去进行替换。Excel中从第二行开始每一行代表一个合同内需要填入的信息。

我们建立的模板和合同信息如下图所示：
![](images\data_preparation.png)
这里有几个注意事项：
1. Excel文档中数字需要改成文本格式，不然像合同编号20190401在写入到word时会变成20190401.0。至于怎么转格式，请参考度娘：https://jingyan.baidu.com/article/ae97a646b3d0b7bbfc461d68.html
2. Excel中的公式需要去除，不然填到word中的信息是公式，而不是值。
3. Word模板中的“【....】”和Excel中的标题必须一一对应，且必须是全中文或全英文字符，因为`python-docx`会将中英混合的内容视为两个及以上的格式（run），导致在替换的时候无法正确识别。
4. Word模板做好后，要用`python-docx`读取一下，看看“【....】”是不是一个独立的run，若不是，则需要从Excel标题栏中重新复制，覆盖word模板中的“【....】”信息，已保证这一串字符是一个run。

In [66]:
import docx #导入docx库
doc = Document("data/合同模板.docx") #打开word文件
for para in doc.paragraphs: #读取word中的每个段落
    for run in para.runs: #读取每个段落中的不同格式（run）
        print(run.text)

合同编号：
【合同编号】
 
【货物名称】
采购
合同
甲方
：
【采购方】
 
乙方
：
ABC商贸
有限责任公司
签订时间：
    
年
  
月
  
日
签订地点
：
甲方和乙方
经过平等协商，在真实、充分地表达各自意愿的基础上，根据《中华人民共和国合同法》的规定，达成如下协议：
 
1.货物的质量标准，按照
行业标准
执行。
2.货物的包装物的供应：包装物随货出售，由乙方负责货物包装物供应。乙方应提供货物运至合同规定的交货地点所需要的包装，以防止货物在转运中损坏或变质。这类包装应采取防潮、防晒、防锈、防腐蚀、防震动及防止其它损坏的必要保护措施，从而保护货物能够经受多次搬运、装卸及长途运输。
二、交货规定
1.交货方法：由乙方送货(国家主管部门规定有送货办法的，按规定的办法执行；没有规定送货办法的，按双方协议执行)；
2.运输方式：由乙方自行选择运输方式，运输及保
险费用由
【运费支付方】
负担。货物交付给甲方之前，货物相关全部风险由乙方承担。
3.交货地点：
  
【交货地点】
  
4.交货日期：乙方应在合同签订
后
3
0
天内完成交货，并附上双方约定的、记录货物相关事项的资料。。
5
.
 
当乙方不能按时交付全部或部分的货物，或者存在这种可能性时，乙方应及时将原因及预定交货日期通知给甲方，并按照甲方的指示，迅速制定必要的对策。
三、验收方法
1.所有货物由乙方送到交货地点且甲方确认收货后
【确认收货天数】
天内，
由甲乙双方共同对货物的包装、外观、数量、商标、型号、规格及性能等进行验收，签署检验报告。如乙方未按约定到甲方指定地点参加检验的，应视为乙方对甲方单方检验的结果予以确认。验收标准执行合同规定的货物质量标准。
如发现乙方所交的货物有任何不符合合同规定之处，应做好记录，
并由双方代表签字，作为甲方向乙方提出维修或退换货的依据。
检验报告仅证明乙方所提供的货物截至出具检验报告之日时可以按合同要求予以接受，但不能视为乙方对货物存在的潜在缺陷所应付的责任的解除。此检验不作为对货物内在质量认定的依据。
2.乙方所提供的货物应充分满足甲方使用的要求,确保供货货物的尺寸、规格、质量符合合同规定，甲方发出的询价函与乙方发出的报价书中规定的内容与合同具有同等的约束力。本合同内的货物质量保证期
为
【质保期】
月，自
验收通过之日起计算。质量保证期间如货物出现

通过以上程序，我们打印显示了合同里面的所有的格式（其中每一行代表一个格式（run））对应的文本（text），我们可以看到“【....】”都是在一行里面的，这样就没问题。由于word版合同里还有一些是在表格里面的，通过`doc.paragraphs`是无法抓取出来的，此时需要用`doc.tables`，表格（tables）里面又包含行(rows)，行还包含单元格(cell)，所以需要读取所有的表格，然后读取所有的行，再读取单元格，并打印显示出来。可见 “【....】” 也是在一行里面的，这样可保证后续替换时可查找到，不会导致遗漏。

In [67]:
for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                print(cell.text)


甲方（盖章）：【采购方】 

法人代表：
或委托代理人：

开户行： 【开户行】
账  号：【账号】
联系人：【联系人】
电  话：【电话】
住  所：【住所】

乙方（盖章）：ABC商贸有限公司

法人代表：
或委托代理人：

开户行：中国建设银行
账  号：989898989898
联系人：张三丰
电  话：999-99999
住  所： 桃花源


Word模板做好后，“【....】”内的信息就不可随意变动了，即便我们将“【合同编号】”里面的“遍”字删掉重新输入，结果还是“【合同编号】”，但此时“【合同编号】”已经不是一个格式了，会变成2个格式。如下示例显示了这个结果,“【合同编号】”已经不在同一行了。所以这个格式非常小气，不可轻易得罪啊！此时需要重新去Excel标题栏复制【合同编号】，再粘贴过去，保存，即可恢复同一格式（也可以在word中复制“【合同编号】”，覆盖粘贴成文本）。

In [72]:
doc = Document("data\合同模板 - 需填入部分格式错误.docx") #打开word文件
for para in doc.paragraphs: #读取word中的每个段落
    for run in para.runs: #读取每个段落中的不同格式（run）
        print(run.text)

合同编号：
【合同编
号】
 
【货物名称】
采购
合同
甲方
：
【采购方】
 


此实例虽然是采购合同，其处理方法适用于所有合同的批量生成，只需要准备好合同的模板，和需要填入合同的信息，剩下的就放心地交给Python吧。合同信息和模板准备好之后，就可开始批量替换，生成合同了。现在跟我出发。

In [74]:
import docx
def info_update(doc,old_info, new_info):
    '''此函数用于批量替换合同中需要替换的信息
    doc:合同模板
    old_info和new_info：原文字和需要替换的新文字
    '''
    #读取段落中的所有run，找到需替换的信息进行替换
    for para in doc.paragraphs: #
        for run in para.runs:
            run.text = run.text.replace(old_info, new_info) #替换信息
    #读取表格中的所有单元格，找到需替换的信息进行替换
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                cell.text = cell.text.replace(old_info, new_info) #替换信息

为方便后续重复调用，以上我们定义了一个函数`info_update()`，它包含三个参数`doc,old_info, new_info`，分别代表word模板，原文本，和新文本。逐个读取word模板中的所有信息，只要遇到原文本，就替换成新文本。然后再读取word中的表格中的信息，也是遇到原文本，就替换成新文本。

In [77]:
from openpyxl import load_workbook #用于读取Excel中的信息
wb = load_workbook('data/合同信息.xlsx')
ws = wb.active
doc = docx.Document("data/合同模板.docx")
for row in range(2, ws.max_row+1):
    for col in range(1, ws.max_column+1):
        #调用上面建立的函数，替换信息
        info_update(doc,str(ws.cell(row=1,column=col).value), str(ws.cell(row=row,column=col).value))
    doc.save("data/{}合同.docx".format(str(ws.cell(row=row,column=3).value)))
    print("{}合同完成".format(str(ws.cell(row=row,column=3).value)))

公司001合同完成
公司002合同完成
公司003合同完成
公司004合同完成
公司005合同完成
公司006合同完成
公司007合同完成
公司008合同完成
公司009合同完成
公司010合同完成


然后使用“openpyxl”库的“load_workbook”模块，读物Excel档的合同信息，遍历每一行，每一列，调用替换信息的函数“info_update”完成合同信息替换，随后保存。

我们以第一份合同为例，逐个看这些步骤是如何完成的。因为Excel中第一行是标题，合同信息是从第二行开始的，所以我们行是从2开始`row in range(2, ws.max_row+1)`，最大行加1结束（因为range函数是取不到最后一个数的，此例中最大行是11，如果不加1，则只能取到10，这样最后一份合同就会被漏掉了）。列也类似，不过是从第一列开始的`col in range(1, ws.max_column+1)`。

第一份合同对应的row值为2，col值为1。原信息是Excel中的标题，对应也就是word中的“【....】”部分。次数原信息先取`ws.cell(row=1,column=1).value`，即如下所示，为'【合同编号】'。因为Excel表中有一些数字，加上str()是为了转换为字符串。

In [78]:
ws.cell(row=1,column=1).value

'【合同编号】'

新信息为`ws.cell(row=2,column=2).value`，如下所示。然后就将word中'【合同编号】'替换为'手机'，再替换第二列，第三列.....直到替换完所有的列，于是第一份合同生成完成，我们使用`doc.save`保存。我们给保存的文件名加上公司名称，以便于区分，公司名是Excel中第三列的值`ws.cell(row=row,column=3).value`。

In [79]:
ws.cell(row=2,column=2).value

'手机'

第一份合同完成后，回到for循环，开始第二份合同的替换和保存，直到搞定所有合同。最终成果如下：
![](images\result.png)